https://github.com/microsoft/recommenders/blob/main/examples/01_prepare_data/data_split.ipynb

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

ml-1m

In [ ]:
column_names = ['MovieID', 'Title', 'Genres']
movies = pd.read_csv("./ml-1m/movies.dat", sep = "::", names = column_names, encoding='latin-1', engine='python')

column_names = ['User_ID', 'Gender', 'Age', 'Occupation', 'Zip-code']
users = pd.read_csv("./ml-1m/users.dat", sep = "::", names = column_names, engine='python')

column_names = ['User_ID', 'MovieID', 'Rating', 'Timestamp']
ratings = pd.read_csv("./ml-1m/ratings.dat", sep = "::", names = column_names, engine='python')
ratings.head()

In [3]:
id2user = dict(enumerate(users.User_ID.unique()))
user2id = {j:i for i, j in id2user.items()}

id2item = dict(enumerate(movies.MovieID.unique()))
item2id = {j:i for i, j in id2item.items()}

In [4]:
ratings['User_ID'] = ratings['User_ID'].map(lambda x: user2id[x]) 
ratings['MovieID'] = ratings['MovieID'].map(lambda x: item2id[x]) 
users['User_ID'] = users['User_ID'].map(lambda x: user2id[x]) 
movies['MovieID'] = movies['MovieID'].map(lambda x: item2id[x]) 

In [5]:
# Aux

In [6]:
users['aux'] = users['Occupation']

In [7]:
movies.Genres = movies.Genres.map(lambda x: x.split('|')[0])
movies[['MovieID', 'Genres']].to_csv('ml_movies.csv', index=False)

In [8]:
users[['User_ID', 'aux']].to_csv('ml_users.csv', index=False)

In [9]:
movies[['MovieID', 'Genres']].Genres.nunique()#, users[['User_ID', 'aux']].aux.nunique()

18

In [10]:
ratings = ratings.sort_values(['User_ID', 'Timestamp'])[['User_ID', 'MovieID']].reset_index(drop=True)

In [11]:
ratings['User_ID'].value_counts().describe()

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
Name: User_ID, dtype: float64

In [12]:
ratings_copy = ratings.copy()

In [13]:
items = ratings.groupby('User_ID')['MovieID'].unique()
test_data = items.map(lambda x: x[-1]).reset_index().values
train_data = ratings_copy.append(pd.DataFrame(test_data, columns=['User_ID', 'MovieID'])).drop_duplicates(keep=False)

/var/folders/ty/d0l26xd500q6hk3ryfccwjy00000gn/T/ipykernel_3732/1184681302.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = ratings_copy.append(pd.DataFrame(test_data, columns=['User_ID', 'MovieID'])).drop_duplicates(keep=False)


In [14]:
item_num = ratings_copy.MovieID.nunique()
item_list = set(range(item_num))
def noninteracted_1(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, len(x))
    return sampled
items.map(noninteracted_1)

User_ID
0       [1192, 2566, 2360, 2903, 3214, 1007, 3542, 288...
1       [2549, 1529, 1174, 52, 3287, 2517, 2029, 1475,...
2       [524, 128, 3621, 1860, 924, 141, 1321, 2756, 2...
3       [585, 736, 1297, 1007, 2584, 286, 639, 2618, 7...
4       [1900, 1004, 3535, 3651, 1463, 3577, 3336, 331...
                              ...                        
6035    [3509, 1702, 3132, 1517, 699, 2543, 3538, 1863...
6036    [2888, 2004, 602, 2332, 3111, 1399, 2042, 2796...
6037    [3308, 805, 3128, 1306, 2013, 1672, 1942, 2226...
6038    [2118, 2140, 3349, 256, 842, 725, 3319, 1809, ...
6039    [1252, 1039, 1810, 298, 2248, 532, 267, 1427, ...
Name: MovieID, Length: 6040, dtype: object

In [15]:
neg_item = items.map(lambda x: x[:-1]).map(noninteracted_1)
train_data['neg_item'] = np.concatenate(neg_item.values)

In [16]:
train_data.to_csv('main_data_train_ml.csv', index=False)

In [17]:
items = pd.concat([train_data.groupby('User_ID')['neg_item'].unique(), ratings.groupby('User_ID')['MovieID'].unique()], axis=1).apply(lambda x: x[0].tolist()+x[1].tolist(), axis=1)

In [19]:
item_num = ratings_copy.MovieID.nunique()
item_list = set(range(item_num))
def noninteracted(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, 99)
    result = [x[-1]] + sampled.tolist()
    return result
res = items.map(noninteracted)

In [20]:
np.save('main_data_test_ml.npy', res.reset_index().values, allow_pickle=True)

In [21]:
ratings.User_ID.max(), ratings.MovieID.max()

(6039, 3882)

imdb

In [2]:
book = pd.read_csv('./archive/Books.csv')
user = pd.read_csv('./archive/Users.csv')
rating = pd.read_csv('./archive/Ratings.csv')

/var/folders/ty/d0l26xd500q6hk3ryfccwjy00000gn/T/ipykernel_2390/1533088594.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('./archive/Books.csv')


In [3]:
ids = rating.groupby('User-ID').ISBN.count().reset_index().query('ISBN > 5')['User-ID'].values
rating = rating.query('`User-ID` in @ids').reset_index(drop=True)
ids = rating['User-ID'].unique()
users = user.query('`User-ID` in @ids').reset_index(drop=True)

books = rating['ISBN'].unique()
items = book.query('ISBN in @books').reset_index(drop=True)
ids = book.ISBN.unique()
rating = rating.query('ISBN in @ids').reset_index(drop=True)

In [4]:
id2user = dict(enumerate(users['User-ID'].unique()))
user2id = {j:i for i, j in id2user.items()}

id2item = dict(enumerate(rating.ISBN.unique()))
item2id = {j:i for i, j in id2item.items()}

In [5]:
rating['User-ID'] = rating['User-ID'].map(lambda x: user2id[x]) 
rating['ISBN'] = rating['ISBN'].map(lambda x: item2id[x]) 
users['User-ID'] = users['User-ID'].map(lambda x: user2id[x]) 
items['ISBN'] = items['ISBN'].map(lambda x: item2id[x]) 

In [6]:
items_ = items.copy()

test

In [7]:
ratings_copy = rating.copy()[['User-ID', 'ISBN']]

In [8]:
items = rating.groupby('User-ID')['ISBN'].unique()
test_data = items.map(lambda x: x[-1]).reset_index().values
train_data = ratings_copy.append(pd.DataFrame(test_data, columns=['User-ID', 'ISBN'])).drop_duplicates(keep=False)

/var/folders/ty/d0l26xd500q6hk3ryfccwjy00000gn/T/ipykernel_2390/1436393466.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = ratings_copy.append(pd.DataFrame(test_data, columns=['User-ID', 'ISBN'])).drop_duplicates(keep=False)


In [9]:
item_num = ratings_copy.ISBN.nunique()
item_list = set(range(item_num))
def noninteracted_1(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, len(x))
    return sampled

In [10]:
neg_item = items.map(lambda x: x[:-1]).map(noninteracted_1)
train_data['neg_item'] = np.concatenate(neg_item.values)

In [11]:
train_data.rename(columns={'User-ID':'User_ID'}).to_csv('main_data_train_ab.csv', index=False)

In [12]:
item_num = ratings_copy.ISBN.nunique()
item_list = set(range(item_num))
def noninteracted(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, 99)
    result = [x[-1]] + sampled.tolist()
    return result
res = items.map(noninteracted)

In [13]:
np.save('main_data_test_ab.npy', res.reset_index().values, allow_pickle=True)

In [14]:
rating['User-ID'].max(), rating.ISBN.max()

(19716, 256725)

aux

In [15]:
users.Age.fillna(users.Age.mode()[0], inplace= True)

def age(x):
    if x < 10:
        return '10대 미만'
    else:
        return str(x)[0]+'0대'

users['Age'] = users.Age.map(age)

In [16]:
users[['User-ID', 'Age']].rename(columns={'User-ID':'User_ID'}).to_csv('ab_users.csv', index=False)

In [17]:
items_[['ISBN', 'Publisher']].to_csv('ab_items.csv', index=False)